In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import json
import glob
import random
import collections

import numpy as np
import dask.dataframe as pd
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
import cv2
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from sklearn.model_selection import train_test_split
import json
import glob
import random
import collections
from glob import glob
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import shutil
import numpy as np
import cv2
from PIL import Image


    
train_df = pd.read_csv("../input/petfinder-pawpularity-score/train.csv")

import pydicom
from fastai.vision.all import *
np.random.seed(20210717)
train_df.head(3)

s_df = pd.read_csv("../input/petfinder-pawpularity-score/sample_submission.csv")
s_df.head(10)

path = ('../input/petfinder-pawpularity-score')
path_train = path + '/train'
path_test = path + '/test'

from glob import glob

train = sorted ( glob( path_train +'/*') )

test= sorted ( glob( path_test +'/*') )
l = len(test)
test[0][-36:-4]

a = []
for i in range (0,l) :
    a.append(test[i][-36:-4])
    
b = []
for i in range (0,l) :
    b.append(38.00)
    
data = []
for elem in zip(a, b):
    data.extend(elem)
data = np.array(data)


In [ ]:
imgdata = []
label = []
for i in train:
    k = cv2.imread(i)
    j = cv2.cvtColor(k, cv2.COLOR_BGR2RGB)
    b = cv2.resize(j, dsize=(80, 80), interpolation=cv2.INTER_CUBIC)
    c = np.array(b)
    imgdata.append(c.reshape(80,80,3)/255.0)
    
x_train = np.array(imgdata)
np.shape(x_train)
    
train_df['Paw'] = round(train_df['Pawpularity']/10)
train_df.head(1)
dfs = train_df['Paw']
y_train = np.array(dfs.astype(float))

from tensorflow.keras import utils as np_utils
from tensorflow.keras.utils import to_categorical
from tensorflow import keras
from tensorflow.keras import layers

#y_train = yt.to_numpy()
np.shape(y_train)

yt = keras.utils.to_categorical(y_train )


X_train, X_test, y_tr, y_test = train_test_split( x_train, yt , shuffle=True, test_size=0.20, random_state=42)


In [ ]:
imgdata = []
label = []
for i in train:
    k = cv2.imread(i)
    j = cv2.cvtColor(k, cv2.COLOR_BGR2RGB)
    b = cv2.resize(j, dsize=(80, 80), interpolation=cv2.INTER_CUBIC)
    c = np.array(b)
    imgdata.append(c.reshape(80,80,3)/255.0)
    
x_train = np.array(imgdata)
np.shape(x_train)
    
train_df['Paw'] = round(train_df['Pawpularity']/10)
train_df.head(1)
dfs = train_df['Paw']
y_train = np.array(dfs.astype(float))

from tensorflow.keras import utils as np_utils
from tensorflow.keras.utils import to_categorical
from tensorflow import keras
from tensorflow.keras import layers

#y_train = yt.to_numpy()
np.shape(y_train)



X_train, X_test, y_tr, y_test = train_test_split( x_train, y_train , shuffle=True, test_size=0.20, random_state=42)


In [ ]:

from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.vgg16 import decode_predictions
from tensorflow.keras.applications.vgg16 import VGG16
# load an image from file



In [ ]:
np.shape(X_train)

In [ ]:
from tensorflow.keras.applications import vgg16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers

import numpy as np
import pandas as pd

from skimage.io import imread
from skimage.transform import resize
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Input, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Dense, Dropout, Activation, Flatten
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


vgg = vgg16.VGG16(include_top=False, weights='imagenet', input_shape=(80,80,3))

output = vgg.layers[-1].output
output =keras.layers.Flatten()(output)
vgg_model = Model(vgg.input, output)

vgg_model.trainable = False
for layer in vgg_model.layers:
    layer.trainable = False

In [ ]:

def get_bottleneck_features(model, input_imgs):
    features = model.predict(X_train, verbose=0)
    return features
    
train_features_vgg = get_bottleneck_features(vgg_model, X_train)

test_features_vgg = get_bottleneck_features(vgg_model, X_test)

In [ ]:
np.shape(train_features_vgg )

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers

model = Sequential()
model.add(vgg_model)
model.add(Dense(512, activation='relu', input_dim=(80,80,3)))
model.add(Dropout(0.3))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-5),
              metrics=['accuracy'])
              
history = model.fit(X_train, y_tr, epochs=1, batch_size=32, validation_split = .3, verbose = 1)   

In [ ]:
imgdata = []
for i in train_features_vgg:
    c = np.array(i)
    imgdata.append(c.reshape(2048,1)/255.0)
    
x_train = np.array(imgdata)
np.shape(x_train)

In [ ]:
import numpy as np
import pandas as pd

from skimage.io import imread
from skimage.transform import resize
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Input, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Dense, Dropout, Activation, Flatten
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

model = models.Sequential()
model.add(layers.Dense(1024, input_shape=(2048,)))
model.add(layers.Dense(256, activation="tanh")) 
model.add(Dropout(0.5))
model.add(layers.Dense(64, activation="tanh"))
model.add(layers.Dense(11, activation="softmax"))
model.summary()


In [ ]:
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization, Activation,Reshape

input_img = Input(shape=(80, 80, 3))
x = Conv2D(16, (3, 3), padding='same')(input_img)
x = Activation('relu')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), padding='same')(x)
x = Activation('relu')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(4, (3, 3), padding='same')(x)
x = Activation('relu')(x)
en = MaxPooling2D((2, 2), padding='same')(x)

en2 = layers.Flatten()(en)

l2 = Dense(64, activation='tanh' ,name='encoder2' ) (en2)

l22 = Dense(400, activation='tanh') (l2)

en3 = layers.Reshape((10,10,4)) (l22)

x = Conv2D(4, (3, 3), padding='same')(en3)
x = Activation('relu')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(8, (3, 3), padding='same')(x)
x = Activation('relu')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(3, (3, 3), padding='same')(x)
decoded = Activation('sigmoid')(x)
model = Model(input_img, decoded)
model.compile(optimizer='adam', loss='binary_crossentropy')
model.summary()
latent_ncdr = Model(input_img, l2) 

In [ ]:
from tensorflow.keras import optimizers
adam = optimizers.Adam(lr=0.001)
model.compile(optimizer=adam, loss='mse')

model.compile(optimizer=adam,loss='mse',metrics=('accuracy'))

model.fit(X_train,X_train, epochs=40, batch_size=32, validation_split = .3, verbose = 0)

In [ ]:
pred = model.predict(X_test)

In [ ]:

decoded_imgs = model.predict(X_test)

n = 10  # how many digits we will display
plt.figure(figsize=(20, 4))
for i in range(n):

    # display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(80,80,3))
    plt.imshow(X_test[i].reshape(80,80,3))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

In [ ]:

decoded_imgs = model.predict(X_test)

n = 10  # how many digits we will display
plt.figure(figsize=(20, 4))
for i in range(n):

    # display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(80,80,3))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

In [ ]:
encoder = Model(inputs=model.input, outputs=model.get_layer('encoder2').output)
encoded_imgs = encoder.predict(X_train)

In [ ]:
encoded = latent_ncdr.predict(X_train)

In [ ]:
test = latent_ncdr.predict(X_test)

In [ ]:
np.shape(encoded_imgs)

In [ ]:
b = []
for i in range (0,len(y_test)) :
    b.append(y_test)
b = np.array(b)

def rmse(y,p) :
    s = 0
    for i in range(0,len(p)):
        o=0
        for j in range(0,10):
            if(y[i][j]==1.0):
                o = j
        
                
        e = abs( (o*10) - p[i] )
        x = e*e
        s=s+x
    w = s/ len(p)
    return np.sqrt(w)
    
z=rmse(y_test,b)
print(z)

In [ ]:
y_test[0][4]==1.0

In [ ]:
imgdata = []
label = []
for i in train:
    k = cv2.imread(i)
    j = cv2.cvtColor(k, cv2.COLOR_BGR2GRAY)
    b = cv2.resize(j, dsize=(100, 100), interpolation=cv2.INTER_CUBIC)
    c = np.array(b)
    imgdata.append(c.reshape(100*100))

x_train = np.array(imgdata)
np.shape(x_train)

dfs = train_df['Pawpularity']
y_train = np.array(dfs.astype(float))

from tensorflow.keras import utils as np_utils
from tensorflow.keras.utils import to_categorical
from tensorflow import keras
from tensorflow.keras import layers

#y_train = yt.to_numpy()
np.shape(y_train)


X_train, X_test, y_tr, y_test = train_test_split( x_train, y_train , shuffle=True, test_size=0.20, random_state=42)


In [ ]:
y_tr=y_tr.astype('float32')/11


In [ ]:
y_tr[6]

In [ ]:
y_test=y_test.astype('float32')/11


In [ ]:
import numpy as np
import pandas as pd

from skimage.io import imread
from skimage.transform import resize
from tensorflow.keras import models
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Input, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Dense, Dropout, Activation, Flatten
import tensorflow as tf
from tensorflow import keras
model = models.Sequential()
model.add(layers.Dense(32, activation="tanh", input_shape=(64,)))
model.add(layers.Dense(1, activation="linear"))
#model.summary()


model.compile(optimizer='Adam',loss='mse',metrics=('accuracy'))

model.fit(encoded, y_tr, epochs=100, batch_size=32, validation_split = .3, verbose = 1)

In [ ]:
pred = np.round(model.predict(test))

In [ ]:
print(y_tr[0],y_test[0])

In [ ]:
a = []
for i in range(0,len(y_test)):
    a.append(.36)
a = np.array(a)
rmse(y_test,a)

In [ ]:
def rmse(a,c):
    b = 0
    for i in range (0,len(a)) :
        j = abs ( a[i] - c[i] )  *10
        k = j * j
        b=b+k
    b= b/len(a)
    return b

In [ ]:
(a,b)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# define used regressor
forest=RandomForestRegressor(n_estimators=8,
                             max_depth=12,
                             criterion='mse',
                             verbose=1,
                            )

# train model
forest.fit(X_train.reshape(len(X_train),80*80*3),y_tr)

In [ ]:
pred = (forest.predict(X_test.reshape(len(X_test),80*80*3)))

rmse(y_test, pred)


In [ ]:
i = 55
print(pred[i],y_test[i])

In [ ]:
pred = (forest.predict(encoder.predict(X_test)))

from sklearn.metrics import accuracy_score


In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train,y_tr)


In [ ]:
pred = (forest.predict(encoder.predict(X_test)))

from sklearn.metrics import accuracy_score


In [ ]:
pred = np.round(clf.predict(X_test))

from sklearn.metrics import accuracy_score
#accuracy_score(y_test, pred)
rmse(y_test, pred)

In [ ]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

model = XGBClassifier()
model.fit(X_train, y_tr)


In [ ]:
pred = np.round(model.predict(X_test))

from sklearn.metrics import accuracy_score
#accuracy_score(y_test, pred)
rmse(y_test, pred)

In [ ]:
from sklearn.linear_model import Perceptron
from sklearn.preprocessing import PolynomialFeatures

X = PolynomialFeatures(interaction_only=True).fit_transform(X_train).astype(int)

clf = Perceptron(fit_intercept=False, max_iter=10, tol=None,shuffle=False).fit(X, y)

In [ ]:
pred = np.round(clf.predict(X_test))

from sklearn.metrics import accuracy_score
#accuracy_score(y_test, pred)
rmse(y_test, pred)